# Modelo de classificação baseado em k-NN

In [18]:
# MODULO 01: CARREGA DADOS
mat_vibration = sio.loadmat('mat_vibration.mat')

variaveis = mat_vibration['mat_vibration'][:, :2560]

y = mat_vibration['mat_vibration'][:, 2560]

numSamp, numVar = variaveis.shape

wavelet = 'NOT'

if wavelet == 'NOT':

    # Particiona conjunto original em Treino e Teste - 70/30
    cv = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
    train_index, test_index = next(cv.split(variaveis, y))

    # Dados de Treino
    Xtrain = variaveis[train_index, :]
    ytrain = y[train_index]

    # Dados de Teste
    Xtest  = variaveis[test_index, :]
    ytest  = y[test_index]





# # Decomposiçao da matriz de descritores com a wavelet escolhida e no nível de decomposição definido.
# import pywt


# wavelet = 'sym'
# nivelmax = pywt.wavelist(wavelet)[6]
# nivel = 3

# if nivel <= nivelmax:

#     for i in range(numSamp):
#         coeffs = pywt.wavedec(variaveis[i,:], wavelet, level=nivel)
#         nCoef = sum([len(c) for c in coeffs])
#         matCoef[i, :nCoef] = np.concatenate(coeffs) 



# Imports

In [2]:
import numpy as np
import scipy.io as sio
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit